<a href="https://colab.research.google.com/github/imbalzy/RecipeQA-FInal-Project-2470/blob/main/2470_final_Bojun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The transformer model part


In [ ]:
import numpy as np
import tensorflow as tf

class Model(tf.keras.Model):
  def __init__(self, window_size, vocab_size):
    super(model, self).__init__()
    
    self.window_size = window_size
    self.batch_size = 1000
    self.embedding_size = 64
    self.optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
    
    #rnn part of the model
    self.word_embedding = tf.Variable(tf.random.truncated_normal([self.english_vocab_size, self.embedding_size], stddev = 0.1))
    self.encoder = tf.keras.layers.LSTM(units = 200, return_sequences=True, return_state=True)
    self.trans_dense = tf.keras.layers.dense(units = 200)

    
    # convolution part of model
    self.conv1 = tf.keras.layers.Conv2D(filters = 16, kernel_size = 3, strides=(1, 1), padding='same')
    self.conv2 = tf.keras.layers.Conv2D(filters = 16, kernel_size = 3, strides=(1, 1), padding='same')
    self.conv3 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, strides=(1, 1), padding='same')
    self.conv4 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, strides=(1, 1), padding='same')
    self.conv5 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, strides=(1, 1), padding='same')
    self.conv6 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, strides=(1, 1), padding='same')
    self.conv_dense = tf.keras.layers.dense(units = 200)

    #combined layers
    self.combined_dense1 = tf.keras.layers.dense(units = 200)
   

   
  @tf.function
  def call(self, question, image):
    question_embedding = tf.nn.embedding_lookup(self.word_embedding, question)

		decoder_seq_output, encoder_final_memory, encoder_final_carry = self.encoder(question_embedding, initial_state = None)
    lstm_output = self.trans_dense(decoder_seq_output)


    image_output = self.conv1(image)
    image_output = self.conv2(image_output)
    image_output = self.conv3(image_output)
    image_output = self.conv4(image_output)
    image_output = self.conv5(image_output)
    image_output = self.conv6(image_output)
    conv_image = self.conv_dense(image_output)

    logits = lstm_output + conv_image
		return logits

  def loss(self, logits, labels):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels, logits))
  
  def accuracy(self, logits, labels):
    correct_predictions = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
